In [1]:
import pandas as pd
import numpy as np
import operator
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
anime = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/irm-final-dataset/anime.csv")
rating = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/irm-final-dataset/rating.csv")

# Phase 1

In [152]:
anime_ratings = pd.DataFrame(rating.groupby('anime_id')['rating'].count())
user_ratings = pd.DataFrame(rating.groupby('user_id')['rating'].count())

In [153]:
anime_ratings = anime_ratings[anime_ratings.rating >= 1000]
user_ratings = user_ratings[user_ratings.rating >= 200]

In [154]:
rating = rating[rating.anime_id.isin(anime_ratings.index.tolist())]
rating = rating[rating.anime_id.isin(user_ratings.index.tolist())]

In [7]:
rating = rating.replace(-1, np.nan)

In [24]:
anime = anime[(anime['type'] == 'TV') | (anime['type'] == 'Movie')]
m = anime['members'].quantile(0.5)
anime = anime[(anime['members'] >= m)]

In [35]:
test_join = anime.merge(reduced_frame, how='inner', on='anime_id')
len(test_join)

1009875

In [37]:
joined = anime.merge(rating, how='inner', on='anime_id')
joined.head()

,anime_id,name,genre,type,episodes,rating_x,members,user_id,rating_y
0,30276,One Punch Man,"Action, Comedy, Parody, Sci-Fi, Seinen, Super ...",TV,12,8.82,552458,3,NaN
1,30276,One Punch Man,"Action, Comedy, Parody, Sci-Fi, Seinen, Super ...",TV,12,8.82,552458,7,8.0
2,30276,One Punch Man,"Action, Comedy, Parody, Sci-Fi, Seinen, Super ...",TV,12,8.82,552458,11,8.0
3,30276,One Punch Man,"Action, Comedy, Parody, Sci-Fi, Seinen, Super ...",TV,12,8.82,552458,17,9.0
4,30276,One Punch Man,"Action, Comedy, Parody, Sci-Fi, Seinen, Super ...",TV,12,8.82,552458,38,9.0


In [43]:
joined = joined[['user_id', 'name', 'rating_y']]
joined = joined[(joined['user_id'] <= 30000)]

pivot = joined.pivot_table(index='user_id', columns='name', values='rating_y')
pivot.head()

name,.hack//G.U. Trilogy,009 Re:Cyborg,00:08,07-Ghost,1000-nen Joou: Queen Millennia,A.D. Police (TV),Abarenbou Rikishi!! Matsutarou,Agatha Christie no Meitantei Poirot to Marple,Ai Shoujo Pollyanna Story,Ai no Wakakusa Monogatari,Ajin,Akatsuki no Yona,Akihabara Dennou-gumi,Aku no Hana,Akuma no Riddle,Alexander Senki,Angel Heart,Anime Himitsu no Hanazono,Aoi Hana,Aoki Densetsu Shoot!,Aoki Hagane no Arpeggio: Ars Nova Cadenza,Appleseed (Movie),Arete Hime,Argento Soma,Aria The Animation,Armitage III: Dual-Matrix,Asobi ni Iku yo!,Bakugan Battle Brawlers: Gundalian Invaders,Bakuman.,Bakuten Shoot Beyblade,Bamboo Blade,Bannou Bunka Neko-Musume (1998),Bartender,Bikini Warriors,Binchou-tan,Bishoujo Senshi Sailor Moon,Bishoujo Senshi Sailor Moon R: The Movie,Bishoujo Senshi Sailor Moon: Sailor Stars,Blade,Bleach Movie 4: Jigoku-hen,...,Toriko 3D: Kaimaku Gourmet Adventure!!,Tsukumo,UN-GO: Inga-ron,Uchuu Kyoudai,Uchuu Senkan Yamato 2,Uchuu Show e Youkoso,Umineko no Naku Koro ni,Waga Seishun no Arcadia,Wagamama High Spec,Wakaba*Girl,Wasurenagumo,Watashi ga Motenai no wa Dou Kangaetemo Omaera ga Warui!,Weiß Kreuz,Weiß Kreuz Glühen,Witch Craft Works,Witch Hunter Robin,Wolverine,Working!!,Working&#039;!!,World Trigger,Yahari Ore no Seishun Love Comedy wa Machigatteiru. Zoku,Yakushiji Ryouko no Kaiki Jikenbo,Yama no Susume,Yami no Matsuei,Yami to Boushi to Hon no Tabibito,Yes! Precure 5 Movie: Kagami no Kuni no Miracle Daibouken!,Yojouhan Shinwa Taikei,Yokuwakaru Gendaimahou,Yume Tsukai,Yume-iro Pâtissière SP Professional,Yuusha-Ou GaoGaiGar,Yuu☆Yuu☆Hakusho,Yu☆Gi☆Oh!: The Dark Side of Dimensions,Z/X: Ignition,Zero no Tsukaima,Zero no Tsukaima F,Zettai Karen Children: The Unlimited - Hyoubu Kyousuke,Zettai Shounen,ef: A Tale of Memories.,xxxHOLiC
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,2.0,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,2.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,5.0,NaN,NaN,6.0,NaN,NaN,8.0,NaN,NaN,7.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
pivot.dropna(axis=0, how='all', inplace=True)

In [45]:
pivot.shape

name,.hack//G.U. Trilogy,009 Re:Cyborg,00:08,07-Ghost,1000-nen Joou: Queen Millennia,A.D. Police (TV),Abarenbou Rikishi!! Matsutarou,Agatha Christie no Meitantei Poirot to Marple,Ai Shoujo Pollyanna Story,Ai no Wakakusa Monogatari,Ajin,Akatsuki no Yona,Akihabara Dennou-gumi,Aku no Hana,Akuma no Riddle,Alexander Senki,Angel Heart,Anime Himitsu no Hanazono,Aoi Hana,Aoki Densetsu Shoot!,Aoki Hagane no Arpeggio: Ars Nova Cadenza,Appleseed (Movie),Arete Hime,Argento Soma,Aria The Animation,Armitage III: Dual-Matrix,Asobi ni Iku yo!,Bakugan Battle Brawlers: Gundalian Invaders,Bakuman.,Bakuten Shoot Beyblade,Bamboo Blade,Bannou Bunka Neko-Musume (1998),Bartender,Bikini Warriors,Binchou-tan,Bishoujo Senshi Sailor Moon,Bishoujo Senshi Sailor Moon R: The Movie,Bishoujo Senshi Sailor Moon: Sailor Stars,Blade,Bleach Movie 4: Jigoku-hen,...,Toriko 3D: Kaimaku Gourmet Adventure!!,Tsukumo,UN-GO: Inga-ron,Uchuu Kyoudai,Uchuu Senkan Yamato 2,Uchuu Show e Youkoso,Umineko no Naku Koro ni,Waga Seishun no Arcadia,Wagamama High Spec,Wakaba*Girl,Wasurenagumo,Watashi ga Motenai no wa Dou Kangaetemo Omaera ga Warui!,Weiß Kreuz,Weiß Kreuz Glühen,Witch Craft Works,Witch Hunter Robin,Wolverine,Working!!,Working&#039;!!,World Trigger,Yahari Ore no Seishun Love Comedy wa Machigatteiru. Zoku,Yakushiji Ryouko no Kaiki Jikenbo,Yama no Susume,Yami no Matsuei,Yami to Boushi to Hon no Tabibito,Yes! Precure 5 Movie: Kagami no Kuni no Miracle Daibouken!,Yojouhan Shinwa Taikei,Yokuwakaru Gendaimahou,Yume Tsukai,Yume-iro Pâtissière SP Professional,Yuusha-Ou GaoGaiGar,Yuu☆Yuu☆Hakusho,Yu☆Gi☆Oh!: The Dark Side of Dimensions,Z/X: Ignition,Zero no Tsukaima,Zero no Tsukaima F,Zettai Karen Children: The Unlimited - Hyoubu Kyousuke,Zettai Shounen,ef: A Tale of Memories.,xxxHOLiC
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,2.0,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,2.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,5.0,NaN,NaN,6.0,NaN,NaN,8.0,NaN,NaN,7.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19993,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN

In [46]:
pivot_norm = pivot.apply(lambda x: x - np.nanmean(x), axis=1)
pivot_norm.fillna(0, inplace=True)

In [47]:
pivot_norm

name,.hack//G.U. Trilogy,009 Re:Cyborg,00:08,07-Ghost,1000-nen Joou: Queen Millennia,A.D. Police (TV),Abarenbou Rikishi!! Matsutarou,Agatha Christie no Meitantei Poirot to Marple,Ai Shoujo Pollyanna Story,Ai no Wakakusa Monogatari,Ajin,Akatsuki no Yona,Akihabara Dennou-gumi,Aku no Hana,Akuma no Riddle,Alexander Senki,Angel Heart,Anime Himitsu no Hanazono,Aoi Hana,Aoki Densetsu Shoot!,Aoki Hagane no Arpeggio: Ars Nova Cadenza,Appleseed (Movie),Arete Hime,Argento Soma,Aria The Animation,Armitage III: Dual-Matrix,Asobi ni Iku yo!,Bakugan Battle Brawlers: Gundalian Invaders,Bakuman.,Bakuten Shoot Beyblade,Bamboo Blade,Bannou Bunka Neko-Musume (1998),Bartender,Bikini Warriors,Binchou-tan,Bishoujo Senshi Sailor Moon,Bishoujo Senshi Sailor Moon R: The Movie,Bishoujo Senshi Sailor Moon: Sailor Stars,Blade,Bleach Movie 4: Jigoku-hen,...,Toriko 3D: Kaimaku Gourmet Adventure!!,Tsukumo,UN-GO: Inga-ron,Uchuu Kyoudai,Uchuu Senkan Yamato 2,Uchuu Show e Youkoso,Umineko no Naku Koro ni,Waga Seishun no Arcadia,Wagamama High Spec,Wakaba*Girl,Wasurenagumo,Watashi ga Motenai no wa Dou Kangaetemo Omaera ga Warui!,Weiß Kreuz,Weiß Kreuz Glühen,Witch Craft Works,Witch Hunter Robin,Wolverine,Working!!,Working&#039;!!,World Trigger,Yahari Ore no Seishun Love Comedy wa Machigatteiru. Zoku,Yakushiji Ryouko no Kaiki Jikenbo,Yama no Susume,Yami no Matsuei,Yami to Boushi to Hon no Tabibito,Yes! Precure 5 Movie: Kagami no Kuni no Miracle Daibouken!,Yojouhan Shinwa Taikei,Yokuwakaru Gendaimahou,Yume Tsukai,Yume-iro Pâtissière SP Professional,Yuusha-Ou GaoGaiGar,Yuu☆Yuu☆Hakusho,Yu☆Gi☆Oh!: The Dark Side of Dimensions,Z/X: Ignition,Zero no Tsukaima,Zero no Tsukaima F,Zettai Karen Children: The Unlimited - Hyoubu Kyousuke,Zettai Shounen,ef: A Tale of Memories.,xxxHOLiC
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.037736,-1.962264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.037736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.037736,NaN,NaN,0.037736,NaN,NaN,-1.962264,0.037736,NaN,0.037736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.037736,NaN,NaN,-2.962264,-2.962264,NaN,NaN,NaN,-1.962264
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.636364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.363636,NaN,0.363636,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.363636,NaN,-2.636364,NaN,NaN,-1.636364,NaN,NaN,0.363636,NaN,NaN,-0.636364,-0.636364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19993,NaN,NaN,NaN,-0.055556,NaN,NaN,NaN,NaN,NaN,NaN,-0.055556,NaN,NaN,NaN,N

In [49]:
user_similarity = pd.DataFrame(cosine_similarity(pivot_norm, pivot_norm),
                               index=pivot_norm.index, columns=pivot_norm.index)
user_similarity.head()

user_id,3,5,7,8,10,11,12,14,16,17,18,19,20,21,23,24,25,26,27,28,29,30,31,32,33,34,37,38,39,40,41,43,44,45,46,47,48,50,51,52,...,19948,19949,19950,19951,19953,19955,19956,19957,19958,19959,19961,19962,19963,19964,19965,19966,19968,19970,19971,19972,19974,19975,19976,19977,19978,19979,19980,19981,19982,19983,19986,19987,19989,19990,19992,19993,19994,19995,19996,20000
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,1.000000,0.039802,0.194479,0.00000,0.0,-0.016144,0.0,0.112419,0.0,0.167891,0.0,0.0,0.0,-0.061890,0.075207,0.000000,0.000000,0.0,0.000000,0.000000,-0.088024,-0.052105,0.000000,0.0,0.0,0.350070,-0.031295,0.055684,0.110262,-0.078672,-0.274833,-0.063046,0.158325,0.000000,7.143661e-02,0.0,0.000000,0.124085,0.021217,0.000000,...,0.045770,0.000000,0.000000,-0.033473,-0.381958,0.0,-0.026728,0.0,0.075339,0.101855,8.424891e-19,0.095639,0.194909,0.043251,0.0,0.147457,0.025175,-0.093008,0.000000,0.040777,-0.079388,0.0,0.202113,-0.017901,0.000000,0.195746,0.092465,0.0,-0.129641,0.091901,0.0,0.000000,-0.135582,0.0,0.0,0.101004,-0.098220,-0.031612,0.0,0.0
5,0.039802,1.000000,-0.050098,-0.13523,0.0,-0.021442,0.0,0.229206,0.0,0.110702,0.0,0.0,0.0,-0.030383,0.222053,-0.113203,0.000000,0.0,0.000000,0.138675,0.316671,0.033546,-0.043033,0.0,0.0,-0.091301,-0.124494,0.124410,-0.040423,-0.036817,-0.100322,0.107313,0.000000,0.000000,-1.440084e-01,0.0,0.045792,0.097568,0.042204,-0.055259,...,-0.022101,0.001200,0.018420,-0.112729,-0.051689,0.0,-0.112221,0.0,0.104986,0.095859,1.385177e-02,0.007737,-0.060093,0.034893,0.0,0.023973,-0.110258,-0.207160,0.012681,0.124940,-0.016603,0.0,0.133771,0.154764,0.041984,0.227458,0.034591,0.0,0.126261,0.109053,0.0,0.059781,0.080507,0.0,0.0,-0.009319,0.033907,-0.038529,0.0,0.0
7,0.194479,-0.050098,1.000000,0.00000,0.0,-0.046298,0.0,0.069262,0.0,0.128234,0.0,0.0,0.0,0.031915,0.034069,0.014564,0.084406,0.0,0.081387,0.000000,-0.070745,-0.078044,0.013691,0.0,0.0,0.115099,-0.024009,0.101373,-0.041086,0.036558,-0.119192,-0.004225,0.017352,0.000000,-2.287347e-02,0.0,0.015791,0.046626,0.000000,0.000000,...,0.053171,-0.100181,-0.033489,-0.113724,-0.125583,0.0,-0.001116,0.0,0.131734,0.145118,4.941759e-02,0.018799,-0.013090,0.115409,0.0,0.063620,-0.041386,-0.081547,-0.099679,-0.050210,-0.127610,0.0,-0.066452,0.100303,0.074530,0.028474,0.061724,0.0,-0.108929,0.210647,0.0,0.052544,-0.102033,0.0,0.0,0.040555,-0.000509,-0.049998,0.0,0.0
8,0.000000,-0.135230,0.000000,1.00000,0.0,0.115294,0.0,-0.238680,0.0,-0.053983,0.0,0.0,0.0,0.000000,-0.358057,0.000000,0.000000,0.0,-0.353553,0.000000,0.000000,0.000000,0.108799,0.0,0.0,0.000000,0.000000,-0.249327,0.000000,0.000000,0.000000,-0.029152,0.000000,0.353553,-8.325433e-18,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.055272,0.0,0.000000,0.067420,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-0.373573,0.000000,-0.097549,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,-0.037915,0.000000,0.0,0.0
10,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [ ]:
anime_index = pd.Series(anime.index, index=anime.name)

In [73]:
user_knn = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(pivot_norm)
user_distances, user_indices = user_knn.kneighbors(pivot_norm)

In [130]:
def get_recommendation(user_id):
    users, similarity = get_similar_user(user_id)
    similarity, users = user_distances[user_id], user_indices[user_id]
    
    if users is None or similarity is None:
        return None
    
    user_arr = np.array([x for x in users[1:]])
    sim_arr = np.array([x for x in similarity[1:]])
    predicted_rating = np.array([])

    
    for anime_name in pivot_norm.columns:
        filtering = pivot_norm.iloc[user_arr][anime_name] != 0.0  
        temp = np.dot(pivot.iloc[user_arr[filtering]][anime_name],
                      sim_arr[filtering]) / np.sum(sim_arr[filtering])
        predicted_rating = np.append(predicted_rating, temp)
    
    temp = pd.DataFrame({'predicted':predicted_rating,
                         'name':pivot_norm.columns})
    
    filtering = (pivot_norm.loc[user_id] == 0.0)

    temp = temp.loc[filtering.values].sort_values(by='predicted',
                                                  ascending=False)

    result = pd.DataFrame(anime.loc[anime_index.loc[temp.name[:10]]])
    return result['name'].tolist()

In [133]:
def print_similar_animes(anime_list):
  for i,element in enumerate(anime_list):
    print('{}. {}'.format(i+1, element))

In [134]:
print_similar_animes(get_recommendation(10))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


1. Natsume Yuujinchou
2. Suzumiya Haruhi no Shoushitsu
3. Digimon Frontier
4. Final Fantasy VII: Advent Children
5. Sankarea
6. Kuroshitsuji II
7. Mononoke
8. K-On!
9. Digimon Adventure
10. Spiral: Suiri no Kizuna


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


# Phase 2

In [31]:
anime['rating'] = anime['rating'].replace(np.nan, 0.0)

In [33]:
anime['rating'].isnull().any()

False

In [34]:
missing = anime.loc[(anime['episodes']=="Unknown") & (anime['type'].isnull())].copy()
missing.shape

(0, 7)

In [35]:
missing

,anime_id,name,genre,type,episodes,rating,members


In [36]:
missing_indexes = missing.index.tolist()
missing_indexes

[]

In [37]:
for i in missing_indexes:
  anime = anime[anime.index != i]

In [38]:
anime.shape

(1534, 7)

In [39]:
anime['genre'].isnull().any()

False

In [40]:
anime['genre'].fillna('Unknown', inplace=True)
anime.isnull().any()

anime_id    False
name        False
genre       False
type        False
episodes    False
rating      False
members     False
dtype: bool

In [ ]:
v = anime['members'].count()
c = anime.rating.mean()
r = anime.rating
m = anime['members'].quantile(0.75)
w = (r*v + c*m)/(v+m)
anime['weighted_rating'] = anime.apply(w, axis=1)

In [41]:
anime_new = anime

In [42]:
anime_new

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
10916,25537,Fate/stay night Movie: Heaven&#039;s Feel,"Action, Fantasy, Magic, Supernatural",Movie,1,0.00,53365
10919,33674,No Game No Life Movie,"Adventure, Comedy, Ecchi, Fantasy, Game, Super...",Movie,1,0.00,32041
10920,33489,Little Witch Academia (TV),"Adventure, Comedy, Fantasy, Magic, School",TV,Unknown,0.00,23398
10976,3786,Evangelion: 3.0+1.0,"Action, Mecha, Sci-Fi",Movie,1,0.00,66600


In [44]:
anime_new.drop(['anime_id', 'rating', 'members', 'episodes'], axis=1,
               inplace=True)
anime_new = pd.concat([anime_new, anime_new['type'].str.get_dummies(),
                       anime_new['genre'].str.get_dummies(sep=',')], axis=1)
anime_new.head()

,name,genre,type,Movie,TV,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Action,Adventure,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Historical,Horror,Josei,Kids,Magic,Mecha,Military,Music,Mystery,Psychological,Romance,School,Sci-Fi,Seinen,Shounen,Slice of Life,Sports
0,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Steins;Gate,"Sci-Fi, Thriller",TV,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
anime_genres = anime_new.loc[:, "Movie":].copy()
anime_genres.head()

,Movie,TV,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Action,Adventure,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Historical,Horror,Josei,Kids,Magic,Mecha,Military,Music,Mystery,Psychological,Romance,School,Sci-Fi,Seinen,Shounen,Slice of Life,Sports
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [51]:
anime_genres = pd.DataFrame(anime_genres)

In [74]:
knn = NearestNeighbors(n_neighbors=12, algorithm='ball_tree').fit(anime_genres)
distances, indices = knn.kneighbors(anime_genres)

In [76]:
def get_index_from_name(name):
    return anime[anime["name"]==name].index.tolist()[0]

In [93]:
def get_recommendation(query):
  listed = []
  found_id = get_index_from_name(query)
  for id in indices[found_id][1:]:
    listed.append(anime.iloc[id]['name'])
  return listed 

In [95]:
def print_similar_animes(listed):
  for i,element in enumerate(listed):
    print('{}. {}'.format(i+1, element))

In [96]:
print_similar_animes(get_recommendation)

1. Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsugu Mono
2. Boruto: Naruto the Movie
3. Naruto: Shippuuden Movie 4 - The Lost Tower
4. Naruto: Shippuuden Movie 6 - Road to Ninja
5. Bleach Movie 3: Fade to Black - Kimi no Na wo Yobu
6. Naruto: Shippuuden
7. Dragon Ball Z Movie 15: Fukkatsu no F
8. Naruto
9. The Last: Naruto the Movie
10. Bleach Movie 4: Jigoku-hen
11. Dragon Ball Z Movie 11: Super Senshi Gekiha!! Katsu no wa Ore da


# Phase 3

In [146]:
anime = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/irm-final-dataset/anime.csv")
# rating = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/irm-final-dataset/rating.csv")

In [103]:
print(anime.shape)
anime = anime[(anime['type'] == 'TV') | (anime['type'] == 'Movie')]
print(anime.shape)

(12294, 7)
(6135, 7)


In [149]:
m = anime['members'].quantile(0.75)
anime = anime[(anime['members'] >= m)]
anime.shape

(3074, 7)

In [155]:
rating.loc[rating.rating == -1, 'rating'] = np.NaN

In [157]:
joined = anime.merge(rating, how='inner', on='anime_id')
joined

,anime_id,name,genre,type,episodes,rating_x,members,user_id,rating_y
0,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679,183,10.0
1,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679,234,10.0
2,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679,288,9.0
3,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679,339,NaN
4,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679,572,10.0
...,...,...,...,...,...,...,...,...,...
1148414,7110,Shounen Maid Kuuro-kun: Tenshi no Uta,"Hentai, Yaoi",OVA,1,5.79,9820,73179,6.0
1148415,7110,Shounen Maid Kuuro-kun: Tenshi no Uta,"Hentai, Yaoi",OVA,1,5.79,9820,73216,8.0
1148416,7110,Shounen Maid Kuuro-kun: Tenshi no Uta,"Hentai, Yaoi",OVA,1,5.79,9820,73247,8.0
1148417,7110,Shounen Maid Kuuro-kun: Tenshi no Uta,"Hentai, Yaoi",OVA,1,5.79,9820,73287,NaN


In [158]:
joined = joined[['user_id', 'name', 'rating_y']]
joined

,user_id,name,rating_y
0,183,Ginga Eiyuu Densetsu,10.0
1,234,Ginga Eiyuu Densetsu,10.0
2,288,Ginga Eiyuu Densetsu,9.0
3,339,Ginga Eiyuu Densetsu,NaN
4,572,Ginga Eiyuu Densetsu,10.0
...,...,...,...
1148414,73179,Shounen Maid Kuuro-kun: Tenshi no Uta,6.0
1148415,73216,Shounen Maid Kuuro-kun: Tenshi no Uta,8.0
1148416,73247,Shounen Maid Kuuro-kun: Tenshi no Uta,8.0
1148417,73287,Shounen Maid Kuuro-kun: Tenshi no Uta,NaN


In [159]:
pivot = joined.pivot_table(index='name', columns='user_id', values='rating_y')

In [160]:
pivot

user_id,3,5,7,8,10,11,12,14,16,17,18,19,20,21,23,24,25,26,27,28,29,30,31,32,33,34,37,38,39,40,41,43,44,45,46,47,48,50,51,52,...,73469,73470,73471,73472,73473,73474,73476,73478,73480,73481,73483,73484,73485,73486,73487,73488,73489,73490,73491,73492,73493,73494,73495,73497,73498,73499,73500,73501,73502,73503,73504,73505,73506,73507,73508,73510,73511,73512,73513,73515
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//G.U. Trilogy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
07-Ghost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,...,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A-Channel+smile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Acchi Kocchi (TV): Place=Princess,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zettai Karen Children: The Unlimited - Hyoubu Kyousuke,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,9.0,7.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zettai Shounen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ef: A Tale of Memories.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
pivot_norm = pivot.apply(lambda x: x - np.nanmean(x), axis=1)
pivot_norm.head()

user_id,3,5,7,8,10,11,12,14,16,17,18,19,20,21,23,24,25,26,27,28,29,30,31,32,33,34,37,38,39,40,41,43,44,45,46,47,48,50,51,52,...,73469,73470,73471,73472,73473,73474,73476,73478,73480,73481,73483,73484,73485,73486,73487,73488,73489,73490,73491,73492,73493,73494,73495,73497,73498,73499,73500,73501,73502,73503,73504,73505,73506,73507,73508,73510,73511,73512,73513,73515
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.656693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//G.U. Trilogy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
07-Ghost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.61692,NaN,NaN,NaN,0.38308,NaN,NaN,NaN,NaN,NaN,...,2.38308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.38308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A-Channel+smile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.403326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Acchi Kocchi (TV): Place=Princess,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
pivot_norm.fillna(0, inplace=True)

In [163]:
pivot_similarity = pd.DataFrame(cosine_similarity(pivot_norm, pivot_norm),
                                index=pivot_norm.index,
                                columns=pivot_norm.index)
pivot_similarity.head()

name,&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,.hack//G.U. Trilogy,07-Ghost,A-Channel+smile,Acchi Kocchi (TV): Place=Princess,Ai no Kusabi (2012),Air in Summer,Ajin,"Akagami no Shirayuki-hime: Nandemonai Takaramono, Kono Page",Akatsuki no Yona,Aku no Hana,Akuma no Riddle,Akuma no Riddle: Shousha wa Dare? Nukiuchi Test,Angel Heart,Anime de Wakaru Shinryounaika,Another: Misaki Mei - Shizuka ni,Aoi Hana,Aoki Hagane no Arpeggio: Ars Nova Cadenza,Appleseed,Appleseed (Movie),Ar Tonelico: Sekai no Owari de Utai Tsuzukeru Shoujo,Arcana Famiglia: Capriccio - stile Arcana Famiglia,Argento Soma,Aria The Animation,Aria The Natural: Sono Futatabi Deaeru Kiseki ni...,Armitage III: Dual-Matrix,Asobi ni Iku yo!,Azumanga Daioh: Gekijou Tanpen,Azumanga Web Daioh,Baka to Test to Shoukanjuu: Matsuri,Baka to Test to Shoukanjuu: Spinout! Sore ga Bokura no Nichijou,Bakugan Battle Brawlers: Gundalian Invaders,Bakuman.,Bakumatsu Kikansetsu Irohanihoheto,Bakuten Shoot Beyblade,Bamboo Blade,Bartender,Batman: Gotham Knight,Bible Black Only Version,Bikini Warriors,...,UN-GO: Inga-ron,Uchuu Kyoudai,Uchuu Show e Youkoso,Umineko no Naku Koro ni,Usagi Drop Specials,Uta no☆Prince-sama♪ Maji Love 2000%: Shining Star Xmas,Vassalord.,Video Girl Ai,Wagamama High Spec,Wakaba*Girl,Wasurenagumo,Watashi ga Motenai no wa Dou Kangaetemo Omaera ga Warui!,Weiß Kreuz,Witch Craft Works,Witch Hunter Robin,Wolverine,Working!!,Working&#039;!!,World Trigger,Yahari Ore no Seishun Love Comedy wa Machigatteiru. Zoku,Yahari Ore no Seishun Love Comedy wa Machigatteiru. Zoku OVA,Yakushiji Ryouko no Kaiki Jikenbo,Yama no Susume,Yami no Matsuei,Yami to Boushi to Hon no Tabibito,Yojouhan Shinwa Taikei,Yokuwakaru Gendaimahou,Yume-iro Pâtissière SP Professional,Yuru Yuri Nachuyachumi!+,Yuusha-Ou GaoGaiGar,Yuu☆Yuu☆Hakusho,Yu☆Gi☆Oh!: The Dark Side of Dimensions,Z/X: Ignition,Zero no Tsukaima,Zero no Tsukaima F,Zettai Karen Children: The Unlimited - Hyoubu Kyousuke,Zettai Shounen,ef: A Tale of Memories.,ef: A Tale of Memories. - Prologue,xxxHOLiC
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,1.000000,0.042919,0.043485,0.047772,0.047952,0.012644,0.076122,0.022031,0.029942,0.030864,0.045751,0.036110,0.027700,0.019009,0.034686,0.024394,0.030372,0.006242,0.032460,0.022246,0.054540,0.014421,0.011187,0.056202,0.085783,0.021021,0.069656,0.050809,0.063899,0.079201,0.063036,0.019254,0.057325,0.042295,0.016504,0.047821,0.018442,0.019646,0.006365,0.013424,...,0.064924,0.046852,0.032674,0.060192,0.046147,0.001756,0.008395,0.007953,0.012586,0.016483,0.060471,0.048703,0.012101,0.036528,0.038458,0.030891,0.074926,0.068530,0.003701,0.040088,0.023217,0.053958,0.064909,0.025115,0.021631,0.030419,0.068169,0.023503,0.028074,0.009068,0.020220,0.013069,0.020972,0.047337,0.033197,0.048739,0.026927,0.051792,0.068119,0.048129
.hack//G.U. Trilogy,0.042919,1.000000,0.044590,0.031470,0.004389,-0.007959,0.058451,0.010907,0.004015,0.018975,0.016262,0.016955,0.016964,0.006721,0.021831,0.016018,0.030448,0.009917,0.023630,0.037759,0.036618,0.009308,0.004756,0.004756,0.049615,0.022615,0.026319,0.024928,0.034180,0.041608,0.027822,0.027014,0.031176,0.024465,0.037397,0.042050,0.001014,0.015495,0.022010,0.007386,...,0.030320,0.002732,0.018759,0.045972,0.021757,0.007806,0.013059,0.000991,0.009353,0.014878,0.015253,0.026731,0.009581,0.023655,0.019437,0.015141,0.020963,0.019409,0.029625,0.011374,0.012538,0.013227,0.019937,0.008441,0.017023,0.022531,0.049933,0.012265,0.019141,0.024665,0.024734,0.000000,0.035263,0.026487,0.027725,0.028840,0.036950,0.047082,0.025093,0.027536
07-Ghost,0.043485,0.044590,1.000000,0.035243,0.043477,0.047082,0.037196,0.034800,0.025354,0.066317,0.008648,0.043075,0.019884,0.023159,-0.000197,0.008125,0.026039,-0.002080,0.018070,0.021999,0.020823,0.039653,0.034664,0.025992,0.022641,-0.003143,0.069416,0.016741,0.014099,0.035551,0.022590,0.028455,0.069436,0.043794,0.025032,0.064541,0.028889,0.017176,0.026382,0.0

In [ ]:
anime_index = pd.Series(anime.index, index=anime.name)

In [192]:
def get_similar_anime(anime_name):
    if anime_name not in pivot_norm.index:
        return None, None
    else:
        similar_animes = pivot_similarity.sort_values(by=anime_name,
                                                      ascending=False).index[1:]
        similar_score = pivot_similarity.sort_values(by=anime_name,
                                                     ascending=False).loc[:,
                                                        anime_name].tolist()[1:]
        return similar_animes, similar_score


In [210]:
def predict_rating(user_id, anime_name):
    animes, scores = get_similar_anime(anime_name)
    anime_arr = np.array([x for x in animes])
    sim_arr = np.array([x for x in scores])
    
    filtering = pivot_norm[user_id].loc[anime_arr] != 0
    
    temp = np.dot(sim_arr[filtering][:10],
               pivot[user_id].loc[anime_arr[filtering][:10]]) \
            / np.sum(sim_arr[filtering][:10])
    
    return temp

In [214]:
def get_recommendation(user_id):
    predicted_rating = np.array([])
    
    for _anime in pivot_norm.index:
        predicted_rating = np.append(predicted_rating,
                                     predict_rating(user_id, _anime))
    
    temp = pd.DataFrame({'predicted':predicted_rating, 'name':pivot_norm.index})
    filtering = (pivot_norm[user_id] == 0.0)
    temp = temp.loc[filtering.values].sort_values(by='predicted',
                                                  ascending=False)

    result = pd.DataFrame(anime.loc[anime_index.loc[temp.name[:10]]])
    return result['name'].tolist()

In [215]:
print_similar_animes(get_recommendation(10))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


1. Rurouni Kenshin: Meiji Kenkaku Romantan - Ishinshishi e no Chinkonka
2. Mimi wo Sumaseba
3. Taboo Tattoo
4. &quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi
5. Ore no Nounai Sentakushi ga, Gakuen Love Comedy wo Zenryoku de Jama Shiteiru
6. Petshop of Horrors
7. Petit Eva: Evangelion@School
8. Persona 3 the Movie 1: Spring of Birth
9. Papa to Kiss in the Dark
10. Papa no Iukoto wo Kikinasai!
